1- Preprocessing :
define characters accepted and tashkeel accepted and remove from the training set any tashkeel and unwanted characters (eg. brackets, numbers... etc)

In [106]:
import re
import numpy as np
import pyarabic.araby as araby
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

In [76]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Habiba
[nltk_data]     ElHussieny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [77]:
tashkeel = ["ْ", "ّ", "ٌ", "ٍ", "ِ", "ً", "َ", "ُ"]
arabicCharacters = "ىعظحرسيشضقثلصطكآماإهزءأفؤغجئدةخوبذتن"
arabicDictionary=['ى', 'ع', 'ظ', 'ح', 'ر', 'س', 'ي', 'ش', 'ض', 'ق', ' ', 'ث', 'ل', 'ص', 'ط', 'ك', 'آ', 'م', 'ا', 'إ', 'ه', 'ز', 'ء', 'أ', 'ف', 'ؤ', 'غ', 'ج', 'ئ', 'د', 'ة', 'خ', 'و', 'ب', 'ذ', 'ت', 'ن']
punctuations = [".", "،", ":", "؛", "-", "؟"]
validCharacters= tashkeel + list(arabicCharacters) + punctuations
charcters_without_tashkeel = list(arabicCharacters) + punctuations

In [78]:
#reading the training dataset
f = open(r"train.txt", "r",encoding="utf-8").read()
print(type(f))

<class 'str'>


In [79]:
print(len(f))
# arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))
#regex to keep arabic letters only and remove any other character (eg. brackets, numbers ...etc)
characters_regex =r'[\s\.\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+'
processedData = re.findall(characters_regex,f)
processedData = " ".join(processedData)
processedData = re.sub(r"\s+"," " ,processedData) #substitute many spaces with one space only
# processedData = ' '.join(word for word in processedData.split() if word not in arabic_stopwords) # remove stopwors from text
print(len(processedData))

18515271
17977409


In [80]:
w = open('processed.txt', 'w', encoding='utf-8')

w.write(processedData)
w.close()

In [81]:
#remove diacritics
without_diacritics= araby.strip_diacritics(processedData)
print(len(without_diacritics))
w2 = open('withoutDiacritics.txt', 'w', encoding='utf-8')

w2.write(without_diacritics)
w2.close()

10692649


In [82]:
#take the procrssed text and separate it into sentences
fileRead=open('processed.txt', encoding='utf-8')
lines=fileRead.readlines()
longString=' '.join(lines)
longStringSplited=re.sub(r"\n", ' ', longString)
#the dot itself got removed in the process
longStringSplited=longStringSplited.split('.')
print(len(longStringSplited))
w2 = open('withoutDiacritics.txt', 'w', encoding='utf-8')

w2.write(without_diacritics)

w3 = open('processed_sentences_separated.txt', 'w', encoding='utf-8')
for line in longStringSplited:
 w3.write(line)

 #now longStringSplited is the list of procrssed text without practices and numbers and dots 
 #the rest of the punctuation still there  


 


40836


In [83]:
# now we want to create a list of words and a crossbonding list of punctuation for each word 

#first remove all puncturation 

fileRead=open('processed_sentences_separated.txt', encoding='utf-8')
lines=fileRead.readlines()
longString=' '.join(lines)
longString=re.sub(r"\n", ' ', longString)
punctuations = ["،", ":", "؛", "-", "؟"]
for element in punctuations:
     longString=re.sub(element, '', longString)



In [84]:
# now the varable called longString has a single string with all the processed words in it 
listOfwordsWith_Diacritics=list()
listOfwordsWith_NoDiacritics=list()


without_diacritics= araby.strip_diacritics(longString)


without_diacritics=re.sub(r"\s+", ' ', without_diacritics)
listOfwordsWith_NoDiacritics=without_diacritics.split(" ")



listOfwordsWith_Diacritics=re.sub(r"\s+", ' ', longString)
listOfwordsWith_Diacritics=listOfwordsWith_Diacritics.split(" ")





In [85]:
#now that we have two separated lists we need to get the diacritics list 


listofDiacritrcs_ToWord=list()
temp=list()
counter=0
flag=0
for word in listOfwordsWith_Diacritics:
    while counter<len(word):
     if word[counter] in arabicDictionary: #checking if the character is a letter
      if (counter+1)<len(word):
        #checking if the next character is also a letter, then that means that the diacritics of the current letter is none so add empty string to the list
        if word[counter +1] in arabicDictionary: 
          temp.append("")
          counter+=1
          continue
      counter+=1 #if it is the end of the word (no more letters) or the next character is a diacritics -> continue looping
      continue
     else:
      if (counter+1)<len(word):
        if word[(counter+1)] not in arabicDictionary: #if the current and the next characters are diacritics, add them together in the list
          temp.append(word[counter]+word[counter+1])
          counter+=2
          continue
      temp.append(word[counter]) #if the current character only is the diacritics add it to the list
      counter+=1    
    listofDiacritrcs_ToWord.append(temp.copy())     
    temp.clear() 
    counter=0
          
          
    
        


print(listOfwordsWith_Diacritics[0:100])
print(listOfwordsWith_NoDiacritics[0:100])

['قَوْلُهُ', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', 'قَالَ', 'الزَّرْكَشِيُّ', 'ابْنُ', 'عَرَفَةَ', 'قَوْلُهُ', 'بِلَفْظٍ', 'يَقْتَضِيه', 'كَإِنْكَارِ', 'غَيْرِ', 'حَدِيثٍ', 'بِالْإِسْلَامِ', 'وُجُوبَ', 'مَا', 'عُلِمَ', 'وُجُوبُهُ', 'مِنْ', 'الدِّينِ', 'ضَرُورَةً', 'كَإِلْقَاءِ', 'مُصْحَفٍ', 'بِقَذَرٍ', 'وَشَدِّ', 'زُنَّارٍ', 'ابْنُ', 'عَرَفَةَ', 'قَوْلُ', 'ابْنِ', 'شَاسٍ', 'أَوْ', 'بِفِعْلٍ', 'يَتَضَمَّنُهُ', 'هُوَ', 'كَلُبْسِ', 'الزُّنَّارِ', 'وَإِلْقَاءِ', 'الْمُصْحَفِ', 'فِي', 'صَرِيحِ', 'النَّجَاسَةِ', 'وَالسُّجُودِ', 'لِلصَّنَمِ', 'وَنَحْوِ', 'ذَلِكَ', 'وَسِحْرٍ', 'مُحَمَّدٌ', 'قَوْلُ', 'مَالِكٍ', 'وَأَصْحَابِهِ', 'أَنَّ', 'السَّاحِرَ', 'كَافِرٌ', 'بِاَللَّهِ', 'تَعَالَى', 'قَالَ', 'مَالِكٌ', 'هُوَ', 'كَالزِّنْدِيقِ', 'إذَا', 'عَمِلَ', 'السِّحْرَ', 'بِنَفْسِهِ', 'قُتِلَ', 'وَلَمْ', 'يُسْتَتَبْ', 'قَوْلُهُ', 'لِعَدَمِ', 'مَا', 'تَتَعَلَّقُ', 'إلَخْ', 'أَيْ', 'الْوَصِيَّةُ', 'قَوْلُهُ', 'مَا', 'مَرَّ', 'أَيْ', 'قُبَيْلَ', 'قَوْلِ', 'الْمَتْنِ', 'لَغَتْ', 'وَلَوْ', 'اقْت

In [114]:
print(len(listOfwordsWith_NoDiacritics))
print(listOfwordsWith_Diacritics[3])
print(listOfwordsWith_NoDiacritics[3])
print(listofDiacritrcs_ToWord[3])
print(" ".join(listofDiacritrcs_ToWord[3]))
print(listofDiacritrcs_ToWord[0:100])

2093191
الْأَوَّلُ
الأول
['', 'ْ', 'َ', 'َّ', 'ُ']
 ْ َ َّ ُ
[['َ', 'ْ', 'ُ', 'ُ'], ['َ', 'ْ'], ['َ', 'َ', 'َ'], ['', 'ْ', 'َ', 'َّ', 'ُ'], ['َ', 'َ', 'ُ'], ['', 'َ', 'ْ'], ['َ', '', 'َ'], ['', '', 'َّ', 'ْ', 'َ', 'ِ', 'ُّ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ', 'ُ'], ['ِ', 'َ', 'ْ', 'ٍ'], ['َ', 'ْ', 'َ', 'ِ', ''], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['َ', 'ْ', 'ِ'], ['َ', 'ِ', '', 'ٍ'], ['ِ', '', 'ْ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ُ', '', 'َ'], ['َ'], ['ُ', 'ِ', 'َ'], ['ُ', 'ُ', '', 'ُ', 'ُ'], ['ِ', 'ْ'], ['', '', 'ِّ', '', 'ِ'], ['َ', 'ُ', '', 'َ', 'ً'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ْ', 'َ', 'ٍ'], ['ِ', 'َ', 'َ', 'ٍ'], ['َ', 'َ', 'ِّ'], ['ُ', 'َّ', '', 'ٍ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ'], ['', 'ْ', 'ِ'], ['َ', '', 'ٍ'], ['َ', 'ْ'], ['ِ', 'ِ', 'ْ', 'ٍ'], ['َ', 'َ', 'َ', 'َّ', 'ُ', 'ُ'], ['ُ', 'َ'], ['َ', 'ُ', 'ْ', 'ِ'], ['', '', 'ُّ', 'َّ', '', 'ِ'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['', 'ْ', 'ُ', 'ْ', 'َ', 'ِ'], ['ِ'], ['َ', 'ِ', '', 'ِ'],

2-Building the LSTM model for character level classifier

In [121]:
for i in range(len(listofDiacritrcs_ToWord)):
        listofDiacritrcs_ToWord[i] = " ".join(listofDiacritrcs_ToWord[i])
# print(listofDiacritrcs_ToWord[0:100])
Y_train= np.array([listofDiacritrcs_ToWord],dtype=object).T
X_train = np.array([listOfwordsWith_NoDiacritics],dtype=object).T
print(str(Y_train.shape))

(2093191, 1)


In [110]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each sentence.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

In [119]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[0]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 2093191, 100)      5000000   
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 2093191, 100)     0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_3 (Dense)             (None, 4)                 404       
                                                                 
Total params: 5,080,804
Trainable params: 5,080,804
Non-trainable params: 0
_________________________________________________________________
None


In [120]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5


ValueError: in user code:

    File "c:\anaconda\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\anaconda\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\anaconda\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\anaconda\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\anaconda\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\anaconda\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "c:\anaconda\lib\site-packages\keras\losses.py", line 152, in __call__
        l